This is the same as Totorial 1 with 2 changes:
1. Looping through - create live video
2. Sematic Segmentation (SS) Camera is used

SS camera labels all object groups with special colours

In [1]:
#all imports
import carla #the sim library itself
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping

In [2]:
# connect to the sim 
client = carla.Client('localhost', 2000)

In [3]:
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

In [4]:
#setting up a car with 2 cameras

# usual - spaw a car and set Autopilot on
vehicle_bp = world.get_blueprint_library().filter('*model3*')
start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
vehicle.set_autopilot(True)

#camera mount offset on the car - you can tweak these to each car to avoid any parts of the car being in the view
CAMERA_POS_Z = 1.6 #this means 1.6m up from the ground
CAMERA_POS_X = 0.9 #this is 0.9m forward

#semantic camera
camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')
camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
camera_sem = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

#normal rgb camera
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')
camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
camera_rgb = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)


def sem_callback(image,data_dict):
    ########## IMPORTANT CHANGE for Semantic camera ##############
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))
    
def rgb_callback(image,data_dict):
    data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = 640
image_h = 360

camera_data = {'sem_image': np.zeros((image_h,image_w,4)),
               'rgb_image': np.zeros((image_h,image_w,4))}

# this actually opens a live stream from the cameras
camera_sem.listen(lambda image: sem_callback(image,camera_data))
camera_rgb.listen(lambda image: rgb_callback(image,camera_data))


In [5]:
while True:
      
    im_h = cv2.hconcat([camera_data['rgb_image'], camera_data['sem_image']])
    cv2.imshow('2 cameras', im_h)
    
    # Break loop if user presses q
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()
camera_sem.stop() # this is the opposite of camera.listen
camera_rgb.stop() 

for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [10]:
# clean up if something went wrong

camera.stop() # this is the opposite of camera.listen
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()